In [1]:
import pandas as pd
import numpy as np
import itertools


In [2]:

xls = pd.ExcelFile("data3.xlsx")
data = pd.read_excel(xls, 'Sheet1')
data2 = pd.read_excel(xls, 'Sheet2')

In [3]:
np_data = np.array(data) # change data with data2 to obtain solution for data2 in sheet2
np_data_copy = np_data.copy()

In [4]:
np_data.tolist()

[[1, 4, 6, 3], [9, 7, 10, 9], [4, 5, 11, 7], [8, 7, 8, 5]]

# Helper Functions

In [5]:
def check_developed(np_array):
    indices = []
    for i in range(np_array.shape[0]):
        for j in range(np_array.shape[1]):
            if np_array[i,j] == 0:
                indices.append((i,j))
                
    

 

     
    com_set = itertools.combinations(indices, np_array.shape[0])
    I = set([i for i in range(np_array.shape[0])])
    #print("I:",I)

    for i in com_set:
        x_set = set()
        y_set = set()
        #print(i)
        for j in range(np_array.shape[0]):
            x_set.add(i[j][0])
            y_set.add(i[j][1])
        #print(x_set,y_set)
        if x_set.issuperset(I) and y_set.issuperset(I):
            return (True,i)
        
        
    return (False,)

In [6]:
def all_covered(di):
    for i in di.values():
        if not i:
            return False
        
    return True

In [7]:
def maxi(di):
    M = 0
    indi = -1
    for i in di:
        if di[i] > M:
            M = di[i]
            indi = i
            
    if indi == -1:
        return -1
    
    
    return indi

In [8]:
def most_freq(key, indi):
    di = {}
    if key in "xX":
        for i in indi:
            if i[0] not in di:
                di[i[0]] = 1
            else:
                di[i[0]] += 1
                
        return maxi(di)
    
    if key in "Yy":
        for i in indi:
            if i[1] not in di:
                di[i[1]] = 1
            else:
                di[i[1]] += 1
                
        return maxi(di)

In [9]:
def decider(indi):
        di_x = {}
        for i in indi:
            if i[0] not in di_x:
                di_x[i[0]] = 1
            else:
                di_x[i[0]] += 1
        di_y = {}
        for i in indi:
            if i[1] not in di_y:
                di_y[i[1]] = 1
            else:
                di_y[i[1]] += 1
                
        if maxi(di_x) == -1 and maxi(di_y) == -1:
            return -1
        elif maxi(di_x) > maxi(di_y):
            return 1
        else:
            return 0

In [10]:
def min_of_uncovered(big_covered, np_array):
    M = 999999
    indi = -1
    for i in range(np_array.shape[0]):
        for j in range(np_array.shape[1]):
            if big_covered[(i,j)] == False and np_array[i,j] < M:
                M = np_array[i,j]
                indi = (i,j)
                
    return M

# Initialize the Matrix by Applying step 1 and step 2

In [11]:
#initialize 
for i in range(np_data.shape[0]):
    mini = np_data[i].min()
#    print(mini)
    for j in range(np_data.shape[1]):
        np_data[i][j]=np_data[i][j]-mini
        
np_data = np_data - np_data.min(axis=0)

In [12]:
np_data.tolist()

[[0, 3, 2, 2], [2, 0, 0, 2], [0, 1, 4, 3], [3, 2, 0, 0]]

# Iterations

In [13]:
print("Data:")
print(np_data_copy)
iteration = 0
print("Iteration 0")
print(np_data)
# checks wheter data is form of optimal or not.
while not check_developed(np_data)[0]:
    #initilizes some variables

    indices = []

    for i in range(np_data.shape[0]):
        for j in range(np_data.shape[1]):
            if np_data[i,j] == 0:
                indices.append((i,j))
                
    covered = {}
    for i in indices:
        covered[i] = False

    lined = {"x":[],"y":[]}
    
    #Tries to cover all 0s with minumum number of lines
    while not all_covered(covered):
        #print("iteration:", iteration)
        k = most_freq("x",indices)
        t = most_freq("y",indices)
        if decider(indices) == 1:
            if k != -1:
                #print(k)
                lined["x"].append(k)
                for i in covered:
                    if i[0] == k:
                        covered[i] = True
                        #print(i)
                        if indices.count(i) > 0:
                            indices.remove(i)
        elif decider(indices) == 0:
            if t != -1:
                #print(t)
                lined["y"].append(t)
                for i in covered:
                    if i[1] == t:
                        covered[i] = True
                        #print(i)
                        if indices.count(i) > 0:
                            indices.remove(i)

        
        
    covered_big = {}
  
    


    for i in range(np_data.shape[0]):
        for j in range(np_data.shape[1]):
            if i in lined["x"] or j in lined["y"]:
                covered_big[(i,j)] = True
            else:
                covered_big[(i,j)] = False
                
                
                
    
    #print("covered_big:",covered_big)
    I = [(i,j) for i in lined["x"] for j in lined["y"]]
    
    mino = min_of_uncovered(covered_big,np_data)
    #print(mino)
    for i in range(np_data.shape[0]):
        for j in range(np_data.shape[1]):
            if (i,j) in I:
                #print(np_data[i,j],"=","np_data[",i,j,"] +",mino)
                np_data[i,j] += mino
            elif covered_big[(i,j)] == False:
                np_data[i,j] -= mino
                #print(np_data[i,j],"=","np_data[",i,j,"] -",mino)
                
    iteration += 1
    print("Iteration:", iteration)
    print(np_data)
                
                
                


Data:
[[ 1  4  6  3]
 [ 9  7 10  9]
 [ 4  5 11  7]
 [ 8  7  8  5]]
Iteration 0
[[0 3 2 2]
 [2 0 0 2]
 [0 1 4 3]
 [3 2 0 0]]
Iteration: 1
[[0 2 1 1]
 [3 0 0 2]
 [0 0 3 2]
 [4 2 0 0]]


# The Results

In [14]:
print("Set of selected")
print(check_developed(np_data)[1])
summ = 0
for i in check_developed(np_data)[1]:
    summ += np_data_copy[i[0],i[1]]
print("The total cost is:", summ)

Set of selected
((0, 0), (1, 2), (2, 1), (3, 3))
The total cost is: 21
